Every Mojo [variable](/mojo/manual/basics/variables.html) provides access to
piece of data (or a "value") stored in memory. As such, to understand how Mojo
helps manage the memory in your program, you need to understand a bit more
about how Mojo treats variables.

Specifically, you need to know that, by default, Mojo handles all variables
using **value semantics**. This means each variable has unique access to a
particular value, and no other variable can modify that value (unless specified
otherwise). For example, consider this code:

In [1]:
x = 1
y = x
y += 1

print(x)
print(y)

1
2


We assign the value of `x` to the `y` variable, and that creates a new value by
copying the value. So when we increment `y`, the value for `x` remains the
same. This is what most people expect and it's an example of value semantics
because each variable handles a unique value. 

However, if, instead of value semantics, a language uses reference semantics,
then `y` actually points to the same data as `x`, so incrementing either one
affects them both.

This behavior applies to functions as well, which makes them intuitive
and predictable. For example, here's the same concept as a function:

In [5]:
def add_one(y):
    y += 1
    print(y)

x = 1
add_one(x)
print(x)

2
1


In this case, we pass `x` into the `add_one()` function, which creates a copy
of the value. Because the function receives a copy, it can modify the value,
and the value in the original `x` variable is unchanged.

If, a language uses "reference semantics," then the function would instead
receive mutable access to the original value. In that case, both `print()`
statements would print `2`.

This is also true when handling types that include data allocated on the heap.
In some languages, if you copy a variable that points to a value on the heap,
the same memory pointer is copied, which means both objects then point to the
same data (this is known as a "shallow copy"). In Mojo, we copy everything,
including the heap-allocated values (this is a "deep copy"). For example:

In [6]:
def update_list(ar):
    ar.append(8)
    print(ar)

array = [2, 4, 6]
update_list(array)
print(array)

[2, 4, 6, 8]
[2, 4, 6]


As we expect from value semantics, the array passed to the function is
completely unique, so when we add a value to the array, the original array is
unchanged.

**Note:** Although Mojo is designed as a superset of Python, this is an example
of how Mojo behaves differently with full value semantics. In Python, this same
code appends new values to the original array, because Python actually passes
all mutable objects by reference. This can be confusing in Python, because
non-mutable values such as integers still behave as value semantics. So the
default argument behavior is more consistent in Mojo.

Notice that the above code uses a `def` function, but remember from the
[section about functions](/mojo/manual/basics/functions.html) that you can also
declare functions with `fn`. A `def` function receives arguments with full
value semantics (via copy, as shown above), whereas, an `fn` function receives
arguments as immutable references. This sounds like a big difference, but the
behavior is still consistent with Mojo's default toward value semantics. That's
because, although an `fn` function receives a reference to the original value
(it can see the original value, not just a copy of it), the function can't
mutate the value—the original variable still has unique ownership of the value.

To demonstrate this, let's create a new function similar to `add_one()` above,
but now using the `fn` declaration:

In [4]:
fn add_two(x: Int):
    # x += 1 # This will cause a compiler error
    var y = x
    y += 2
    print(y)

x = 1
add_two(x)
print(x)

3
1


In this case, because the `fn` function receives an **immutable** reference, the
function cannot change the value of `x`. We must instead make a copy of it
ourselves and then update the new variable. Again, this works the way we expect
with value semantics: the new `y` variable holds a value that's completely
unique from the original `x` variable.

The way the `fn` function receives the `x` value is sort of a "look but don't
touch" approach to value semantics. This is also a more memory-efficient
approach when dealing with memory-intensive arguments, because the argument
value is not copied unless we explicitly decide to copy it. (This is just
one example of how using `fn` instead of `def` can make your program more
performant.)

So, that's the gist of how Mojo defaults to value semantics. However, Mojo is
designed with great flexibility, and there are plenty of good reasons why you
might want a function to receive a **mutable** reference. Mojo allows you to
do that, but it also helps ensure that you do so safely (avoiding errors such as
use-after-free, double-free and memory leaks).

On the next page, you'll see how to modify the argument semantics, allowing
functions to receive a mutable reference to a value, and how take full
ownership of a value (both with zero copies and full memory safety).